# NYC Case Study

## Data Importing and inspection 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Basic RDD openration") \
    .getOrCreate()

In [2]:
# Creating a new DataFrame by calling read method on Spark object
df = spark.read.format("csv").load('/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', header=True, InferSchema= True)
df

DataFrame[Summons Number: bigint, Plate ID: string, Registration State: string, Issue Date: timestamp, Violation Code: int, Vehicle Body Type: string, Vehicle Make: string, Violation Precinct: int, Issuer Precinct: int, Violation Time: string]

In [3]:
df.show(2)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
only showing top 2 rows



In [4]:
df.columns

['Summons Number',
 'Plate ID',
 'Registration State',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Precinct',
 'Issuer Precinct',
 'Violation Time']

In [5]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)



## Questions 

## Examine the Data 

### 1)  Find the total number of tickets for the year.

In [6]:
df.count()

#We can see there are over 10 million issued in the year 2017

10803028

### 2)  Find out the number of unique states from where the cars that got parking tickets came.

In [7]:
df.groupBy('Registration State').count().show()

+------------------+------+
|Registration State| count|
+------------------+------+
|                AZ| 26426|
|                SC| 21836|
|                NS|   730|
|                LA|  3466|
|                MN| 18227|
|                NJ|925965|
|                MX|     4|
|                DC|  4250|
|                OR|  5483|
|                99| 36625|
|                NT|     4|
|                VA| 72626|
|                RI| 12224|
|                KY|  3833|
|                WY|   411|
|                BC|   127|
|                NH|  8752|
|                MI| 15703|
|                NV|  1846|
|                GV|   704|
+------------------+------+
only showing top 20 rows



##  Aggregation tasks

### 1)  How often does each violation code occur? Display the frequency of the top five violation codes.

In [8]:
from pyspark.sql.functions import col
df.cube('Violation Code').count().sort(col("count").desc()).show(5)

#Violation code 21 has the highest occurence closely followed by code 36

+--------------+--------+
|Violation Code|   count|
+--------------+--------+
|          null|10803028|
|            21| 1528588|
|            36| 1400614|
|            38| 1062304|
|            14|  893498|
+--------------+--------+
only showing top 5 rows



### 2)  How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? 

In [9]:
df.cube('Vehicle Body Type').count().sort(col("count").desc()).show(5)

#We can see that SUBN has the highest parking ticket count followed by 4DSD

+-----------------+--------+
|Vehicle Body Type|   count|
+-----------------+--------+
|             null|10803028|
|             SUBN| 3719802|
|             4DSD| 3082020|
|              VAN| 1411970|
|             DELV|  687330|
+-----------------+--------+
only showing top 5 rows



In [10]:
df.cube('Vehicle Make').count().sort(col("count").desc()).show(5)

#We can see that FORD has the highest parking ticket count followed by TOYOTA

+------------+--------+
|Vehicle Make|   count|
+------------+--------+
|        null|10803028|
|        FORD| 1280958|
|       TOYOT| 1211451|
|       HONDA| 1079238|
|       NISSA|  918590|
+------------+--------+
only showing top 5 rows



###  A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following

In [11]:
df.cube('Violation Precinct').count().sort(col("count").desc()).show(6)

+------------------+--------+
|Violation Precinct|   count|
+------------------+--------+
|              null|10803028|
|                 0| 2072400|
|                19|  535671|
|                14|  352450|
|                 1|  331810|
|                18|  306920|
+------------------+--------+
only showing top 6 rows



In [12]:
df.cube('Issuer Precinct').count().sort(col("count").desc()).show(6)

+---------------+--------+
|Issuer Precinct|   count|
+---------------+--------+
|           null|10803028|
|              0| 2388479|
|             19|  521513|
|             14|  344977|
|              1|  321170|
|             18|  296553|
+---------------+--------+
only showing top 6 rows

